In [4]:
from langchain_community.vectorstores import FAISS
from configparser import ConfigParser

In [5]:
#load config
config = ConfigParser()
config.read('config.ini')

['config.ini']

In [6]:
import csv
from langchain.schema import Document

documents = []
with open("data.csv", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for row_index, row in enumerate(reader):
        index_id = row.get("index_id", f"row_{row_index}")
        field_value = row.get("description", "").strip()
        if field_value:  # 確保不是空的
            doc = Document(
                page_content=field_value,
                metadata={
                    "index_id": index_id,
                    "field": "description"
                }
            )
            documents.append(doc)

print(f"產生了 {len(documents)} 筆 description 文件")
print(documents[0])


產生了 679 筆 description 文件
page_content='泰式打拋醬原先的微酸微辣口感，讓人食慾大開。蒜香、辣椒與空心菜快炒後，清脆的空心菜口味層次豐富，是一道非常下飯的泰式料理。' metadata={'index_id': '28', 'field': 'description'}


In [7]:
index_map = {}
with open("data.csv", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        index_id = row["index_id"]
        index_map[index_id] = row

In [8]:
from langchain_openai import AzureOpenAIEmbeddings

embedding_model = AzureOpenAIEmbeddings(
    openai_api_version=config["AzureOpenAI"]["VERSION"],
    azure_deployment=config["AzureOpenAI"]["Embedding_DEPLOYMENT_NAME"],
    azure_endpoint=config["AzureOpenAI"]["ENDPOINT"],
    api_key=config["AzureOpenAI"]["KEY"],
)

In [9]:
# db = FAISS.from_documents(documents, embedding_model)
# 載入 FAISS index
db = FAISS.load_local("faiss_db", embedding_model, "cuisines", allow_dangerous_deserialization=True)

In [10]:
# 儲存 FAISS index
db.save_local("faiss_db", "cuisines")

In [443]:
query = "我想吃清爽的雞肉料理？"

In [444]:
result = db.similarity_search_with_score(query, 10)

In [445]:
# 收集所有檢索結果
saved_results = []

for i, (doc, dist) in enumerate(result):
    index_id = doc.metadata["index_id"]
    field = doc.metadata["field"]
    content = doc.page_content
    original_row = index_map.get(index_id, {})

    result_entry = {
        "Result_Rank": i + 1,
        "Score": round(dist, 4),
        "Matched_Field": field,
        "Field_Content": content,
        "Index_ID": index_id,
        "Name": original_row.get("name", ""),
        "Description": original_row.get("description", ""),
        "Ingredients": original_row.get("ingredients", ""),
        "Instructions": original_row.get("instructions", "")
    }

    saved_results.append(result_entry)


In [446]:
contexts = []
for i, result in enumerate(saved_results):
    context = f"Result Rank: {result['Result_Rank']}\n"
    context += f"Score: {result['Score']}\n"
    context += f"Matched Field: {result['Matched_Field']}\n"
    context += f"Field Content: {result['Field_Content']}\n"
    context += f"Index ID: {result['Index_ID']}\n"
    context += f"Name: {result['Name']}\n"
    context += f"Description: {result['Description']}\n"
    context += f"Ingredients: {result['Ingredients']}\n"
    context += f"Instructions: {result['Instructions']}\n\n"

    contexts.append(context)

In [447]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    openai_api_version=config["AzureOpenAI"]["VERSION"],
    azure_deployment=config["AzureOpenAI"]["GPT4o_DEPLOYMENT_NAME"],
    azure_endpoint=config["AzureOpenAI"]["ENDPOINT"],
    api_key=config["AzureOpenAI"]["KEY"],
)

In [448]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

# 封裝成 List[Document]
documents = [Document(page_content=c) for c in contexts]

# 建立 prompt
prompt = ChatPromptTemplate.from_template(
"""
Answer the following question based only on the provided context,
besides the answer, please also provide a list of comparison, 
just list item, not table.

{context}

Question: {input}
請用繁體中文回答
"""
)

# 建立 chain 並傳入
document_chain = create_stuff_documents_chain(llm, prompt)

llm_result = document_chain.invoke(
    {
        "input": query,
        "context": documents
    }
)

In [449]:
for result in saved_results:
    print(f"Rank: {result['Result_Rank']}, Score: {result['Score']}")
    print(f"Matched Field: {result['Matched_Field']}")
    print(f"Field Content: {result['Field_Content']}")
    print(f"Index ID: {result['Index_ID']}")
    print(f"Name: {result['Name']}")
    print(f"Description: {result['Description']}")
    print(f"Ingredients: {result['Ingredients']}")
    print(f"Instructions: {result['Instructions']}")
    print("-" * 40)

Rank: 1, Score: 0.802299976348877
Matched Field: description
Field Content: 運動完最適合來道爽口不膩的涼拌菜，撲鼻而來的油蔥香味，讓人食慾大開；悶熟的雞胸吃起來口感嫩而不柴，剝絲後更增添入口的嚼勁，配上番茄自帶的甜酸風味及玉米的清甜，整道料理吃起來口感非常多元且又高蛋白低熱量，很適合運動、減醣的人食用。
Index ID: 193
Name: 涼拌蔥油雞絲
Description: 運動完最適合來道爽口不膩的涼拌菜，撲鼻而來的油蔥香味，讓人食慾大開；悶熟的雞胸吃起來口感嫩而不柴，剝絲後更增添入口的嚼勁，配上番茄自帶的甜酸風味及玉米的清甜，整道料理吃起來口感非常多元且又高蛋白低熱量，很適合運動、減醣的人食用。
Ingredients: ['青花椰菜 150公克', '牛蕃茄 1顆顆', '雞胸肉 200公克', '玉米粒(罐頭) 85公克', '鹽 些許公克', '金黃蔥油 10毫升', '鮮磨黑胡椒 些許公克']
Instructions: ['雞胸肉去皮，其他食材清洗後，分切成便於入口大小', '起一鍋水，水滾後倒入一匙蔥油；然後關火放入雞胸肉(視雞胸肉厚度，約悶10-12分鐘)', '待雞胸肉悶熟後，取出', '重新開火，水滾後放入其他食材，燙熟後取出', '用叉子將雞胸肉刮成雞絲狀', '將所有食材裝盤後，撒上鹽、胡椒、蔥油調味']
----------------------------------------
Rank: 2, Score: 0.8166999816894531
Matched Field: description
Field Content: 想要美味又想吃得清爽無負擔，可以試試以水炒取代油炒，不僅能吃到美味的蒜香雞翅，又能為身體帶來好油，是道令人允指的美味佳餚！
Index ID: 100
Name: 水炒蒜香雞翅
Description: 想要美味又想吃得清爽無負擔，可以試試以水炒取代油炒，不僅能吃到美味的蒜香雞翅，又能為身體帶來好油，是道令人允指的美味佳餚！
Ingredients: ['水 320毫升', '青蔥 2根', '紅蘿蔔 1根', '雞翅(三節) 8支', '米酒 30毫升', '鹽 7公克', '醬油 15毫升', '辣椒 1根', '老薑 

In [450]:
print(llm_result)

根據提供的內容，以下是幾款清爽的雞肉料理選擇：

1. **涼拌蔥油雞絲**
2. **鹹水雞**
3. **蔥油舞菇拌雞胸**
4. **香煎雞胸彩椒炒藜麥飯**

### 比較清單：
- **涼拌蔥油雞絲**：高蛋白低熱量，搭配番茄和玉米的甜酸清爽。
- **鹹水雞**：炎炎夏日適合，帶濃郁蔥香，肉質Q彈清涼爽口。
- **蔥油舞菇拌雞胸**：蔥油香氣與舞菇口感結合，搭配清爽蔬菜。
- **香煎雞胸彩椒炒藜麥飯**：搭配彩椒和藜麥飯，咖哩粉增香，低熱量又健康。
